Getfemで領域ごとに質量行列を設定できたため、方法を共有する。

In [1]:
import getfem as gf
import numpy as np

今回は簡単のため、１次元２要素のモデルで作成を行う。

In [2]:
m = gf.Mesh('cartesian', np.arange(3))
print m


BEGIN POINTS LIST

  POINT  0  0
  POINT  1  1
  POINT  2  2

END POINTS LIST



BEGIN MESH STRUCTURE DESCRIPTION

CONVEX 0    'GT_PK(1,1)'      0  1
CONVEX 1    'GT_PK(1,1)'      1  2

END MESH STRUCTURE DESCRIPTION



In [3]:
mfu = gf.MeshFem(m, 1)
mfu.set_fem(gf.Fem('FEM_PK(1,1)'))
mfd = gf.MeshFem(m, 1)
mfd.set_fem(gf.Fem('FEM_PK(1,1)'))
print mfu
print mfd


BEGIN MESH_FEM

QDIM 1
 CONVEX 0 'FEM_PK(1,1)'
 CONVEX 1 'FEM_PK(1,1)'
 BEGIN DOF_ENUMERATION 
  0:  0 1
  1:  1 2
 END DOF_ENUMERATION 
END MESH_FEM


BEGIN MESH_FEM

QDIM 1
 CONVEX 0 'FEM_PK(1,1)'
 CONVEX 1 'FEM_PK(1,1)'
 BEGIN DOF_ENUMERATION 
  0:  0 1
  1:  1 2
 END DOF_ENUMERATION 
END MESH_FEM



In [4]:
mim = gf.MeshIm(m, gf.Integ('IM_GAUSS1D(1)'))
print mim


BEGIN MESH_IM

 CONVEX 0 'IM_GAUSS1D(1)'
 CONVEX 1 'IM_GAUSS1D(1)'
END MESH_IM



ここからが、本題。メッシュに領域を設定する。今回CONVEXは２個なので、２個定義する。

In [5]:
m.set_region(1, 0)
m.set_region(2, 1)
print m


BEGIN POINTS LIST

  POINT  0  0
  POINT  1  1
  POINT  2  2

END POINTS LIST



BEGIN MESH STRUCTURE DESCRIPTION

CONVEX 0    'GT_PK(1,1)'      0  1
CONVEX 1    'GT_PK(1,1)'      1  2

END MESH STRUCTURE DESCRIPTION
BEGIN REGION 1
0 
END REGION 1
BEGIN REGION 2
1 
END REGION 2



"CONVESX 0"と"CONVEX 1"に対してそれぞれの要素行列のみ作成できれば、異なる物性値を定義してそれを後から重ね合わせられる。まずは全領域(-1)で作成してみる。

In [6]:
gf.asm_mass_matrix(mim, mfu, mfu, -1).full()

array([[ 0.25,  0.25,  0.  ],
       [ 0.25,  0.5 ,  0.25],
       [ 0.  ,  0.25,  0.25]])

次に、領域1に対して作成する。

In [7]:
gf.asm_mass_matrix(mim, mfu, mfu, 1).full()

array([[ 0.25,  0.25,  0.  ],
       [ 0.25,  0.25,  0.  ],
       [ 0.  ,  0.  ,  0.  ]])

領域2も同様に作成できる。

In [8]:
gf.asm_mass_matrix(mim, mfu, mfu, 2).full()

array([[ 0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.25,  0.25],
       [ 0.  ,  0.25,  0.25]])

以上のように各領域に対して行列を作成できた。後は異なる質量密度を掛け重ね合わせれば良い。

ついでなので、剛性行列もやってみる。

In [9]:
clambda = np.repeat(1.0, m.nbpts())
mu = np.repeat(1.0, m.nbpts())
gf.asm_linear_elasticity(mim, mfu, mfu, clambda, mu, -1).full()

array([[ 3., -3.,  0.],
       [-3.,  6., -3.],
       [ 0., -3.,  3.]])

In [10]:
gf.asm_linear_elasticity(mim, mfu, mfd, clambda, mu, 1).full()

array([[ 3., -3.,  0.],
       [-3.,  3.,  0.],
       [ 0.,  0.,  0.]])

In [11]:
gf.asm_linear_elasticity(mim, mfu, mfd, clambda, mu, 2).full()

array([[ 0.,  0.,  0.],
       [ 0.,  3., -3.],
       [ 0., -3.,  3.]])